import pandas as pd

In [1]:
!pip install aitextgen

     |████████████████████████████████| 572 kB 8.0 MB/s 
     |████████████████████████████████| 3.1 MB 53.3 MB/s 
     |████████████████████████████████| 87 kB 8.8 MB/s 
     |████████████████████████████████| 524 kB 68.2 MB/s 
     |████████████████████████████████| 829 kB 66.1 MB/s 
     |████████████████████████████████| 329 kB 37.5 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
     |████████████████████████████████| 132 kB 71.3 MB/s 
     |████████████████████████████████| 1.1 MB 68.9 MB/s 
     |████████████████████████████████| 61 kB 631 kB/s 
     |████████████████████████████████| 3.3 MB 59.6 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 271 kB 64.3 MB/s 
     |████████████████████████████████| 160 kB 74.5 MB/s 
     |████████████████████████████████| 192 kB 74.6 MB/s 
  Created wheel for aitextgen: filename=aitextgen-0.5.2-py3-none-any.whl size=575905 sha256=53fd6fabca981e9f1e723adb998157445f1d187f7f7

In [9]:
import json

api_token = {"username":"courageekoh","key":"83bc06bfc6490721d52067847e849603"}

!mkdir .kaggle
!mkdir ~/.kaggle

with open('/content/kaggle.json', 'w') as file:
  json.dump(api_token, file)

!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘.kaggle’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
!pip install kaggle
!kaggle --version

Kaggle API 1.5.4


In [16]:
!kaggle datasets download -d courageekoh/code-to-docstring-finetune-encoder-decoder

 99% 1.56G/1.57G [00:13<00:00, 115MB/s]
100% 1.57G/1.57G [00:13<00:00, 125MB/s]


In [17]:
!unzip code-to-docstring-finetune-encoder-decoder.zip -d data 

Archive:  code-to-docstring-finetune-encoder-decoder.zip
  inflating: data/encoder_decoder/checkpoint-42000/config.json  
  inflating: data/encoder_decoder/checkpoint-42000/optimizer.pt  
  inflating: data/encoder_decoder/checkpoint-42000/pytorch_model.bin  
  inflating: data/encoder_decoder/checkpoint-42000/rng_state.pth  
  inflating: data/encoder_decoder/checkpoint-42000/scaler.pt  
  inflating: data/encoder_decoder/checkpoint-42000/scheduler.pt  
  inflating: data/encoder_decoder/checkpoint-42000/special_tokens_map.json  
  inflating: data/encoder_decoder/checkpoint-42000/tokenizer.json  
  inflating: data/encoder_decoder/checkpoint-42000/tokenizer_config.json  
  inflating: data/encoder_decoder/checkpoint-42000/trainer_state.json  
  inflating: data/encoder_decoder/checkpoint-42000/training_args.bin  
  inflating: data/encoder_decoder/runs/Nov29_21-39-19_4276d342c1ee/1638221963.9234145/events.out.tfevents.1638221963.4276d342c1ee.27.1  
  inflating: data/encoder_decoder/runs/Nov29_

df = pd.read_csv('lichess_elite.csv')

df.sample(1_000_000).to_csv('lichess_elite_sampled.csv')

df

## Loading the libraries

In [20]:
import transformers
import pandas as pd

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

import numpy as np

from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4


from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional
from tqdm.notebook import tqdm

import os

In [41]:
!pip show tokenizers

Name: tokenizers
Version: 0.10.3
Summary: Fast and Customizable Tokenizers
Home-page: https://github.com/huggingface/tokenizers
Author: Anthony MOI
Author-email: anthony@huggingface.co
License: Apache License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: transformers


In [46]:
from transformers import RobertaTokenizerFast, RobertaForCausalLM, RobertaConfig
import torch

path = os.path.join(os.getcwd(),"/content/data/encoder_decoder/checkpoint-42000")

tokenizer = RobertaTokenizerFast.from_pretrained(path)
model = EncoderDecoderModel.from_pretrained(path)
model.to('cuda')

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [35]:
path

'/content/data/encoder_decoder/checkpoint-42000'

In [28]:
path

'/content/data/encoder_decoder/checkpoint-42000'

In [27]:
def calc_smooth_bleu(ref, hyp):
    return bleu([ref], hyp, smoothing_function=smoothie)

In [31]:
data = pd.read_csv('encode_deode_bleu.csv')
data['predicted'] = ''
data['bleu_score'] = ''

In [32]:
data.head()

,input,target,predicted,bleu_score
0,"summarize: def write_comment(fh, comment):\n ...",Writes a comment to the file in Java propertie...,,
1,"summarize: def register_post_execute(self, fun...",Register a function for calling after code exe...,,
2,"summarize: def frames_to_samples(frames, hop_l...",Converts frame indices to audio sample indices.,,
3,"summarize: def partition(condition, collection...",Partitions a list into two based on a condition.,,
4,"summarize: def _get_axis_mode(self, axis):\n ...",will get the axis mode for the current series,,


In [33]:
data['input'][0]

'summarize: def write_comment(fh, comment):\n  """\n    \n  """\n  _require_string(comment, \'comments\')\n  fh.write(_escape_comment(comment))\n  fh.write(b\'\\n\')'

In [48]:
for i in tqdm(range(len(data)), total = len(data)):
    p = data['input'][i]

    input_ids = tokenizer.encode(p, return_tensors='pt').cuda()

    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=50, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=3
    )

    a = []

    for idx, sample_output in enumerate(sample_outputs):
        a.append(tokenizer.decode(sample_output, skip_special_tokens=True))

    ref = [' '.join(data['target'][i].split())]
    pred = [[' '.join(j.split())] for j in a]

    bleu_scores = []

    for k in pred:
      score = calc_smooth_bleu(ref[0], k[0])*100
      bleu_scores.append(score)

    data['predicted'][i] = pred[np.argmax(bleu_scores)][0]
    data['bleu_score'][i] = max(bleu_scores)

  0%|          | 0/1001 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [50]:
data

,input,target,predicted,bleu_score
0,"summarize: def write_comment(fh, comment):\n ...",Writes a comment to the file in Java propertie...,s to and the a an of for from in_.].]])))) ) p...,12.3961
1,"summarize: def register_post_execute(self, fun...",Register a function for calling after code exe...,"=_ a. thes to an of with licies None get this,...",9.703
2,"summarize: def frames_to_samples(frames, hop_l...",Converts frame indices to audio sample indices.,=_ a. thesname to for of in all default Retrie...,7.61272
3,"summarize: def partition(condition, collection...",Partitions a list into two based on a condition.,=_ a. the to allsname for in not( ( this an of...,13.8349
4,"summarize: def _get_axis_mode(self, axis):\n ...",will get the axis mode for the current series,s and for the a an of to from in_.().].'].').'...,10.9254
...,...,...,...,...
996,"summarize: def url_value_preprocessor(self, f)...",Registers a function as URL value preprocessor...,"s to a the an of for and or from in_.().(),())...",19.8944
997,summarize: async def parse_vn_results(soup):\n...,Parse Visual Novel search pages.,='=_ a.s thename to for of anRetrieve(('(_ ( a...,9.46519
998,"summarize: def generate(secret, age, **payload...",Generate a one-time jwt with an age in seconds,"s the a an of to for and or from in_.().(),())...",9.62507
999,summarize: def char_tokens_to_char_offsets(si_...,Convert character ``Offset``s to character ran...,"=_ a. the to allsname and ( of in,' -: not( se...",4.74968


In [49]:
!nvidia-smi

Sat Dec  4 12:43:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    35W / 250W |   3437MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [51]:
data.to_csv('code_to_text_enc_dec.csv', index = False)

## Random Test Code

In [ ]:
p = 'summarize: def _enable_autopx(self):\n        """\n        """\n        # override run_cell\n        self._original_run_cell = self.shell.run_cell\n        self.shell.run_cell = self.pxrun_cell\n\n        self._autopx = True\n        print "%autopx enabled"'

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(p, return_tensors='pt') #.to('cuda')

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
)

pred = []



print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    pred.append([tokenizer.decode(sample_output, skip_special_tokens=True)])

In [ ]:
pred

In [ ]:
sample_output